利用python进行数据分析

## 第五章 pandas入门

### 两种数据形式的介绍

<span class="mark">书123-164</span>

<span class="burk">Series类型</span>

1.利用pd.Series()创建，df.values(array类型) 以及df.index调用

2.利用布尔数对df、series进行过滤df[df>0]

3.利用dict创建Series,即df=pd.Series({'a':1,'b':2,'c':3})#这里的键是指的行索引，这是与建立DataFrame不同的方式->他是通过列索引作为键key

4.利用isnull()和notnull()判断df、series是否有空值

5对于Series的加减，如果有一方的行为NaN，那么加减后就是空

6.series.index.name就是行索引那一列的名称


<span class="mark">DataFrame类型</span>

1.通过dict来创建df，其中值可以用list、series、array来表示

2.series添加入dataframe中,df['debt']=series,对齐行索引，当不存在行索引时，则为空

3.df['a']=df.b=='duiqi'返回一个布尔值

4.del df['a']可以移除a这一列

5.嵌套字典创建df，{'a':{2001:2,2001:3},'b':{2000:1.5,2001:1.7}}  其中外部字典的键作为列，内部字典的键作为行索引。

6.创建index()类型，pd.index(np.arange(3))

7.索引对象的方法和属性：append,difference(差),intersection(交),union(并),isin(是否在),delete,drop,insert,unique.



In [2]:
import pandas as pd
a=pd.DataFrame({'年龄':[2,2,3],
              '身高':[4,5,6]},index=[2001,2002,2003])
a

,年龄,身高
2001,2,4
2002,2,5
2003,3,6


### 基本功能

df.reindex(<span class="mark">index,method</span>,<span class="burk">fill_value</span>,limit,tolerance,level,copy)有ffill、bfill的填充方式

In [13]:
#1.reindex重建索引：改变行索引、列索引
a.reindex([2001,2002,2004])
#当没有行索引时，就为空值
a.reindex([2001,2002,2004],method='ffill')#2004的值由2003的值填充补齐

,年龄,身高
2001,1,4
2002,2,5
2004,3,6


In [22]:
#2.drop删除条目
a.drop(2001)
a.drop([2001,2002])
a.drop('年龄',axis=1)
a.drop(['年龄','身高'],axis=1)
#也可以加上inplace=True来进行操作

""
2001
2002
2003


In [76]:
#3.索引、选择与过滤
a[a>2]
a.loc[:2002]#还有对索引号的iloc
a.isin([2,4])

,年龄,身高
2001,False,True
2002,True,False
2003,False,False


In [37]:
#4.算术和数据对齐
s1=pd.Series([7.3,-2.5,3.4,1.5],index=['a','c','d','e'])
s2=pd.Series([-2.1,3.6,-1.5,4,3.1],index=['a','c','e','f','g'])
s1+s2#对于没有的行索引，则用NaN来填充
#同理如果列索引没有对齐，则还是用NaN来填充

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

In [42]:
#5.填充缺失值
s1.add(s2,fill_value=0)#如果不对齐的则加0即可

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

<span class="mark">函数应用和映射</span>

In [48]:
#6.apply函数
a
a.apply(lambda x:x.max()-x.min())#以axis=0生成series
a.apply(lambda x:x.max()-x.min(),axis=1)#以axis=1生成series
#applymap函数格式化输出,针对每个元素的
a.applymap(lambda x: '%.2f' %x)
#但是对于map函数,只能映射一列的

,年龄,身高
2001,1.00,4.00
2002,2.00,5.00
2003,3.00,6.00


In [52]:
#7.排序
a.sort_index()
a.sort_index(axis=1,ascending=False)
a.sort_values(by=['身高','年龄'],ascending=False)#当有缺失值时，会把缺失值排放在最后面
#对于a.rank(method='first')专门针对数值相同下还需要排序，first指把第一次出现的值放在前面
#但是我认为像rank这种函数用处不大，处理这种问题一般用groupby分组,或者多次sort_values

,年龄,身高
2003,3,6
2002,2,5
2001,1,4


In [54]:
#8.is_unique判断index索引是否唯一
a.index.is_unique

True

In [70]:
#9.idxmax用于DataFrame中，生成一个Series,返回每列中最大数的索引。
a.idxmax()
#对比而言,argmax用于Series和array中，生成索引
a['年龄'].argmax()

2

<span class="mark">value_counts实现多列统计</span>

In [81]:
#10.使用多列的value_counts()
a.apply(pd.value_counts).fillna(0)#行索引是df出现的所有值，而value是每个值出现的次数

,年龄,身高
2,2.0,0.0
3,1.0,0.0
4,0.0,1.0
5,0.0,1.0
6,0.0,1.0


## 第六章、数据载入、存储及文件格式

### 文本数据的读写

read_csv,read_table读取文件

1.当第一行无变量名称，无表头,那么read_csv('xxx',header=None),当然也可以自己设置列索引名和行索引名,即read_csv('xxx',name=[xxx列名称])

2.当然也可以直接指定某列为行索引,read_csv('xxx',index_col='xx')

多重索引('分层索引'):read_csv('xxx',index_col=['xx1','xx2'])#当然你也可以通过set_index(['xx1','xx2'])来

3.当文本不是以逗号分隔，可能是不规则的分隔，像\n,之类的：可以用手工处理；也可以向<span class="mark">read_table传入一个正则表达式作为分隔符'\s+'</span>

即<span class="mark">read_table('xxx',sep='\s+')</span>

4.read_csv中的skiprows读取时跳过某些行,read_csv('xxx',skiprows=[0,2,3])#跳过0，2，3行

总结：有用的参数对于read_csv\table

1.delimtier\sep分隔符

2.header

3.index_col设置行索引

4.names当无表头时，自己设置表头

5.parse_dates将数据解析为datetime

6.encoding作为编码'utf-8'


### 分块读入文本数据

1.可以通过设置pd.set_option('display.max_rows',10)

2.也可以在read_csv('xxx',nrows=10)中设置

3.还可以设置read_csv中的chunksize,分块读入文件，chunksize为每一块的行数

### 将数据写入文本格式to_csv

1.当不想保存的文件以逗号形式分割,则引入sep，即to_csv('xxx',sep='|')

2.对保存的文件的缺失值进行标注,引入na_rep,即to_csv('xxx',na_rep='NULL')

3.如果<span class="mark">不想让行列标签保存</span>，即to_csv('xxx',index=False,header=False)

4.只想保存特定的列，即to_csv('xxx',index=False,columns=['a','b','c'])

### JSON数据(javascript object notation)

定义：是Web浏览器和其他应用间通过HTTP请求发送数据的标准格式。基本类型是对象(字典)、数组（列表）、字符串、数字、布尔值和空值。

In [6]:
#import json
#result = json.loads(obj)就导入文件，即json字符串转化为python形式
#asjson=json.dumps(result)将python对象转换回json

In [12]:
#当读入json后，变量为这样
result={'a':[{'q':10,'w':'dds','e':['r','t']},
       {'q':22,'w':'dd','e':['y','u']}]}

In [13]:
pd.DataFrame(result['a'],columns=['q','w'])#因为result['a']是一个列表，即每一个元素作为dataframe的一行，而这个元素是个字典，
#我们引入columns作为这个字典的key，那么对应的这一行变成了字典的values

,q,w
0,10,dds
1,22,dd


<span class="mark">pd.read_json</span>可以自动把json数据集转化为Series或DataFrame，即pd.read_json('xxx')

导出\保存json文件，即data.to_json

### 读取excel文件

pd.read_excel('xxx','sheet1')前者是excel的路径,后者是这个excel哪个sheet

当然你也可以直接读完excel,即read_excel('xxx',sheet_name=None)，然后生成的是一个<span class="mark">字典</span>，你可以通过以<span class="mark">sheet名称作为key</span>，来调用每个<span class="mark">sheet的值</span>

保存为excel形式，即df.to_excel('xxx')

### Web与API交互

In [19]:
import requests
url='https://api.github.com/repos/pandas-dev/pandas/issues'
resp=requests.get(url)
data=resp.json()
data#生成了json形式的数据文件
pd.DataFrame(data,columns=['number','title','labels','state'])

,number,title,labels,state
0,49198,BUG: panda._testing.assert_interval_array_equa...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
1,49197,Vendored klib quadatric probing,"[{'id': 8935311, 'node_id': 'MDU6TGFiZWw4OTM1M...",open
2,49196,BUG: result_type=expand seems to behave exactl...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
3,49195,BUG: Pandas crashes when asked to display a da...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
4,49194,RLS: pandas 1.5.2,"[{'id': 131473665, 'node_id': 'MDU6TGFiZWwxMzE...",open
...,...,...,...,...
25,49131,BUG: groupby reorders categorical categories,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
26,49130,QST: DF row fields with strings containing si...,"[{'id': 34444536, 'node_id': 'MDU6TGFiZWwzNDQ0...",open
27,49118,BUG: Return type discrepancy in USFederalHolid...,"[{'id': 53181044, 'node_id': 'MDU6TGFiZWw1MzE4...",open
28,49116,ENH: tz_convert with non-hour shifts from UTC,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open


## 第七章 数据清洗与准备

### 处理缺失值

In [2]:
import pandas as pd
import numpy as np
data=pd.DataFrame([[1,6.5,3],[1,np.nan,np.nan],
                  [np.nan,np.nan,np.nan],[np.nan,6.5,3]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [4]:
data.dropna()
data.dropna(how='all')#行全是nan删除
data.dropna(axis=1,how='all')#列全是nan删除

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [10]:
data.dropna(thresh=1)#丢弃1个缺失值的行
data.dropna(thresh=2)#丢弃2个缺失值的行
data.dropna(thresh=3)#丢弃3个缺失值的行

,0,1,2
0,1.0,6.5,3.0


In [25]:
data.fillna(0)
data.fillna({1:0.5,2:0})#字典来填充，key代表dataframe的列索引号，value代表填充的值
data.fillna(method='ffill')#把前面的值填充后面的
data.fillna(method='ffill',limit=1)#每列只填充1行的缺失值
data.fillna(data.mean())#用每列的均值填充

,0,1,2
0,1.0,6.5,3.0
1,1.0,6.5,3.0
2,1.0,6.5,3.0
3,1.0,6.5,3.0


### 数据转换

#### 删除重复列

In [26]:
data=pd.DataFrame({'k1':['one','two']*3+['two'],
                  'k2':[1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [31]:
data.duplicated()#行完全相同，才会出现True
data.k1.duplicated()
data.drop_duplicates()#只删除行完全相同的
data.drop_duplicates(['k1'])
data.drop_duplicates(['k1','k2'],keep='last')#k1和k2共同完全相同，保留最后一个出现的重复行

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
6,two,4


#### 使用函数或映射

In [35]:
#使用函数或映射进行数据转换
data=pd.DataFrame({'food':['bacon','pulled pork','bacon','Pastrami','corned beef','Bacon','pastrami','honey ham','nova lox'],
                  'ounces':[4,3,12,6,7.5,8,3,5,6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [45]:
#如果想用一列映射，则设置一个字典+map
lowercased=data['food'].str.lower()#这里是一个series类型，无法直接lower,要识别每一个元素才行
data['animal']=lowercased.map({'bacon':'pig','pulled pork':'pig','pastrami':'cow','corned beef':'cow',
                              'honey ham':'pig','nova lox':'salmon'})#新增的一列映射列

#或者传一个函数，合并解决上面两个
data['food'].map(lambda x: {'bacon':'pig','pulled pork':'pig','pastrami':'cow','corned beef':'cow',
                              'honey ham':'pig','nova lox':'salmon'}[x.lower()])#series中传入每一个元素，所以不用str

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

#### 替代值

In [46]:
#运用replace函数进行替代，其实和map函数替代的功能差不多
data = pd.Series([1,-999,2,-999,-1000,3])
data

0       1
1    -999
2       2
3    -999
4   -1000
5       3
dtype: int64

In [49]:
data.replace(-999,np.nan)#单个值替换
data.replace([-999,-1000],np.nan)#也可以多个值替换为一个值

data.replace([-999,-1000],[np.nan,0])#也可以多个值对应替换多个值
data.replace({-999:np.nan,-1000:0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

#### 重命名轴索引

In [52]:
data=pd.DataFrame(np.arange(12).reshape((3,4)),index=['Ohio','Colorado','New York'],columns=['one','two','three','four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [57]:
data.index=data.index.map(lambda x : x[:4].upper())#映射至每一个元素
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


rename函数同样可以对索引行列名进行修改

In [59]:
data.rename(index=str.title,columns=str.upper)#把index的首字母大写，其余都小写
data.rename(index={'OHIO':'INDIANA'},columns={'three':'peekaboo'})#用字典的形式也可以,个人喜欢还是用map针对某行/列进行映射

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


#### 离散化和分箱

In [72]:
#用cut(根据最大值和最小值的等分间隔判定),qcut(根据数据的分位数进行分割)
ages=[20,22,25,27,21,23,37,31,61,45,41,32]
cats=pd.cut(ages,bins=[18,25,35,60,100])#将数据分成(18,25),(25,35),(35,60),(60,100)，还可以加入right进行区间封闭
#注意:返回的是一个categorical类型
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [73]:
cats.codes#返回区间的分类标签
cats.categories#返回区间的划分情况
cats.value_counts()#返回每个区间，样本所占的个数

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

In [75]:
#可以将区间的值，进行赋予标签处理
group_names=['Youth','YoungAdult','MiddleAged','Senior']
pd.cut(ages,bins=[18,25,35,60,100],labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [76]:
data=np.random.rand(20)
pd.cut(data,4,precision=2)#将数据从最大值和最小值的差距等分4份

[(0.74, 0.96], (0.51, 0.74], (0.51, 0.74], (0.74, 0.96], (0.058, 0.29], ..., (0.058, 0.29], (0.058, 0.29], (0.74, 0.96], (0.51, 0.74], (0.74, 0.96]]
Length: 20
Categories (4, interval[float64]): [(0.058, 0.29] < (0.29, 0.51] < (0.51, 0.74] < (0.74, 0.96]]

In [84]:
#qcut分位数切割
pd.qcut(data,4)#那么应该是0,0.25,0.5,0.75,1分位数
pd.qcut(data,4).value_counts()

pd.qcut(data,q=[0,0.1,0.5,0.9,1])#传入的bins是分位数****
pd.qcut(data,q=[0,0.1,0.5,0.9,1]).value_counts()

(0.0581, 0.147]    2
(0.147, 0.638]     8
(0.638, 0.832]     8
(0.832, 0.965]     2
dtype: int64

#### 检测和过滤异常值

<span class="mark">any这个函数是关键的条件筛选</span>

In [91]:
data=pd.DataFrame(np.random.randn(1000,4))
col=data[2]#获取第二列的数据
col[np.abs(col)>3]#筛选这一列中大于3

383    3.401371
864    3.080381
996   -4.390542
Name: 2, dtype: float64

any() 函数用于判断给定的可迭代参数 iterable 是否全部为 False，则返回 False，<span class="mark">如果有一个为 True，则返回 True。</span>

元素<span class="mark">除<span class="mark">了是 0、空、False</span> 外</span>都算 True。

In [100]:
data[(np.abs(data)>3)]
data[(np.abs(data)>3).any(1)]#只要有一个绝对值大于3，则返回True
#all完全相反,必须都为True，才返回True

,0,1,2,3
237,3.307244,-0.172445,-0.080066,0.105755
281,0.537795,-0.327163,0.333047,-3.646631
373,-0.689321,-1.407191,0.287250,-3.644404
383,-1.395569,-1.285358,3.401371,0.485149
497,3.262049,-0.659592,-1.204811,-0.157917
531,3.614380,0.082388,-0.131493,1.184431
696,-0.902959,-0.668019,0.078587,3.113322
864,-0.562028,0.057214,3.080381,0.498871
996,0.268756,0.127939,-4.390542,-0.256801


np.sign(data)只要<span class="mark">data大于0,则返回1</span>；<span class="mark">小于0,则返回-1</span>

In [105]:
np.sign(data)
data[np.abs(data)>3]=np.sign(data)*3#把绝对值大于3的值变成-3或者3,
data[(data==3).any(1)]#查看被赋值的

,0,1,2,3
237,3.000000,-0.172445,-0.080066,0.105755
383,-1.395569,-1.285358,3.000000,0.485149
497,3.000000,-0.659592,-1.204811,-0.157917
531,3.000000,0.082388,-0.131493,1.184431
696,-0.902959,-0.668019,0.078587,3.000000
864,-0.562028,0.057214,3.000000,0.498871


#### 置换和随机抽样

np.random.permutation对<span class="mark">Series进行随机排序</span>,有点shuffle

In [117]:
df=pd.DataFrame(np.arange(5*4).reshape((5,4)))
sampler=np.random.permutation(5)#会有返回值,但shuffle无
sampler

array([3, 4, 0, 2, 1])

In [124]:
#permutation+take的使用
df.take(sampler)#take默认axis=0,取了sampler随机排列的索引号

df.sample(n=3)#随机抽取3行，不重复的
df.sample(n=3,replace=True)#当然加上replace也可以重复抽取行

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
1,4,5,6,7


#### 虚拟变量+离散化取值

In [160]:
df=pd.DataFrame({'key':['b','b','a','c','a','b'],
                'data1':range(6)})
dummies=pd.get_dummies(df['key'],prefix='key')#prefix可以加上前缀
#运用join函数把两个dataframe拼接起来，有点concat作用
df1=df[['data1']].join(dummies)
df1

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [163]:
values=np.random.rand(10)
bins=[0,0.2,0.4,0.6,0.8,1]
pd.get_dummies(pd.cut(values,bins))#将离散化的数据虚拟化

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,1,0,0,0,0
1,0,0,0,1,0
2,0,0,0,1,0
3,0,0,1,0,0
4,0,1,0,0,0
5,0,1,0,0,0
6,0,1,0,0,0
7,0,0,1,0,0
8,0,0,0,0,1
9,0,0,0,0,1


### 字符串操作

#### 字符串对象方法

In [16]:
#split和strip经常一起用于分隔字符串并清除空格
val='a,b, guido'
pieces=[x.strip() for x in val.split(',')]

#使用join函数把list转化为str类型
'::'.join(pieces)

#使用in,index,find函数检测子字符串
'guido' in val
val.index('guido')#返回第一个字符串的索引
val.find('guido')#和上面那个是一样的，只不过如果不存在查询的字符串，则返回-1

#count计算字符串出现次数
val.count(',')

#replace将旧字符串替换为新字符串
val.replace(',','::')

'a::b:: guido'

#### 正则表达式regex
re模块有三个主题：匹配、替代、拆分

In [53]:
#描述一个或多个空白字符的正则表达式是'\s+'
import re
text='foo  bar\t baz  \tqux'
re.split('\s+',text)#一步就好了

#写完整点就是这样子
regex=re.compile('\s+')#先让模块编译，需要识别空白字符
regex.split(text)#再用这个模块分隔文本


['foo', 'bar', 'baz', 'qux']


['foo', 'bar', 'baz', 'qux']

In [27]:
#匹配:findall、match、search
#如果想获得一个所有匹配正则表达式的列表(即空白字符)
regex.findall(text)

#match是只在字符串的起始位置进行匹配
#search返回的仅仅是第一个匹配项

text='''Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
'''
pattern=r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'#编译了上面所有的邮件
regex=re.compile(pattern,flags=re.IGNORECASE)
print(regex.findall(text))#返回所有匹配的项

regex.search(text)#只返回第一个匹配项

print(regex.match(text))#只在字符串的起始位置进行匹配

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']
None


In [29]:
#替代:sub函数
print(regex.sub('REDACTED',text))#把刚才编译的东西替换了

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [34]:
#升级版，如果想把刚才的地址进行修改，分层更加清楚,(用户名),(域名),(域名后缀)
pattern=r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'#用元组的形式
regex=re.compile(pattern,flags=re.IGNORECASE)

#用match查询
m=regex.match('wesm@bright.net')
m.groups()

#还可以用findall查询返回的元组列表
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

#### 向量化字符串函数

In [47]:
import pandas as pd
data={'Dave':'dave@google.com','Steve':'steve@gmail.com','Rob':'rob@gmail.com','Wes':np.nan}
data=pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [48]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [49]:
#也可以结合re模块使用

print(data.str.findall(pattern,flags=re.IGNORECASE))

data.str.match(pattern,flags=re.IGNORECASE)

data.str[:5]#对字符串切片


Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object


Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

## 第八章 数据规整:连接、联合与重塑

### 分层索引

In [6]:
import numpy as np
import pandas as pd
data=pd.Series(np.random.randn(9),index=[['a','a','a','b','b','c','c','d','d'],[1,2,3,1,3,1,2,2,3]])
data

a  1   -0.748269
   2   -1.435420
   3    0.123231
b  1   -1.148087
   3   -0.818621
c  1    1.440558
   2   -0.450273
d  2   -2.273846
   3    0.066395
dtype: float64

In [79]:
data['b':'c']#外层切片
from pandas import IndexSlice as idx
data.loc[idx['a':'b',1:2]]#内层+外层的切片，如果还需要调用不同的列，那么就是data.loc[idx['a':'b',1:2],'能源']

data['b']#外层索引，如果想把外层的索引号加上，用双[]就行了
data.loc[['b','d']]#多个外层索引
data.loc[:,2]#双重索引，内层+外层

data.unstack()#把内层索引变成列索引
data.unstack().stack()#把列索引变成内层索引

a  1   -0.748269
   2   -1.435420
   3    0.123231
b  1   -1.148087
   3   -0.818621
c  1    1.440558
   2   -0.450273
d  2   -2.273846
   3    0.066395
dtype: float64

#### 重新排序和层级排序

In [85]:
frame=pd.DataFrame(np.arange(12).reshape((4,3)),index=[['a','a','b','b'],[1,2,1,2]],columns=[['Ohio','Ohio','Colorado'],['Green','Red','Green']])
frame.index.names=['key1','key2']
frame.columns.names=['state','color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [90]:
frame.swaplevel('key1','key2')#交换内层和外层的行索引
frame.sort_index(level=1)#level代表用第几个行索引，0代表外层，1代表内层第一个,依次顺序...
frame.swaplevel(0,1).sort_index(level=0)#仍然交换内层行索引,并以外层索引排序

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

#### 按层级进行汇总统计

In [93]:
frame.sum(level='key2')#按照同一key2下，进行加总,比如：当key2=1时，Green为0+6，Red为1+7，Green2+8

frame.sum(level='color',axis=1)#同理还可以改变轴，对列进行合并

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 联合与合并数据集

#### merge连接

In [94]:
#pd.merge(df1,df2)#按照相同列合并
#pd.merge(df1,df2,on='key')按照哪个相同的列合并
#pd.merge(df3,df4,left_on='lkey',right_on='rkey')左边的表格按照哪个连接，右边的表格按照哪个连接
#pd.merge(df1,df2,how='outer')两个表格中的行无对应时，外连接保留他们,并用NAN填充
#pd.merge(df1,df2,on='key',how='left')左连接，保留左边的行，丢弃右边不对应的行
#pd.merge(df1,df2,how='inner')丢弃两边行不对应的
#pd.merge(left,right,on=['key1','key2'],how='outer')#两个先后顺序的主键
#pd,merge(left,right,on='key',suffixes=('_left','_right'))处理重叠的列名

#### join连接

In [95]:
#left2.join(right2,how='outer')
#left2.join(right2,on='key')默认以左连接,保留左边的

#### concat沿轴向连接

In [96]:
s1=pd.Series([0,1],index=['a','b'])
s2=pd.Series([2,3,4],index=['c','d','e'])
s3=pd.Series([5,6],index=['f','g'])

In [106]:
pd.concat([s1,s2,s3])#默认以o轴
pd.concat([s1,s2,s3],axis=1)#默认是以外连接
s4=pd.concat([s1,s3])
pd.concat([s1,s4],axis=1,join='inner')#以内连接

,0,1
a,0,0
b,1,1


In [111]:
#创建多层索引
pd.concat([s1,s2,s3],keys=['one','two','three'])#以0轴外连接后，将keys上的作为外层索引
pd.concat([s1,s2,s3],axis=1,keys=['one','two','three'])#以1轴连接,keys赋予列索引
#还有个ignore_index=True当concat的时候

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


### 重塑和透视

#### 使用多层索引进行重塑

In [112]:
#data.unstack(0)#拆外层索引
#data.unstack('state')#拆索引列为'state'的
#unstack默认拆内层索引
#data.unstack('state'.stack('side'))

#### 透视

In [ ]:
#data.pivot(index,column,value)=====等价于data.unstack(column)的方法

#### 逆透视(处理成面板数据)

In [113]:
#data.melt(id_vars=['index'],value_vars=['column'],value_name='能源消费总量')=====等价于data.stack(column) 

## 第十章 数据聚合与分组操作

### groupby机制
重采样

In [276]:
import pandas as pd
import numpy as np
df=pd.DataFrame({'key1':['a','a','b','b','a'],
                'key2':['one','two','one','two','one'],
                'data1':np.random.randn(5),
                'data2':np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.334590,-0.424050
1,a,two,-0.570007,0.005418
2,b,one,0.782883,-0.567793
3,b,two,-0.000837,0.338568
4,a,one,-0.634305,-1.035988


In [6]:
#groupby后不会返回数据
#先分组后聚合均值
df['data1'].groupby(df['key1']).mean()#mean聚合函数自动过滤除float、int类型的字段
#针对两个变量分组，聚合
df['data1'].groupby([df['key1'],df['key2']]).mean().unstack()#聚合后，并把内层索引变成列索引

key2,one,two
key1,,
a,0.120544,0.894465
b,0.952106,0.925075


In [7]:
#还可以在外部自定义一个相等长度的变量，进行groupby
states=np.array(['Ohio','California','California','Ohio','Ohio'])
years=np.array([2005,2005,2006,2005,2006])
df['data1'].groupby([states,years]).mean()

California  2005    0.894465
            2006    0.952106
Ohio        2005    0.226769
            2006    0.712625
Name: data1, dtype: float64

In [10]:
#count和size的区别是：size只返回一列,而count可以返回多列
df.groupby(['key1','key2']).count()
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

#### 遍历个分组

In [15]:
#groupby会返回一个包含组名(分组变量名称)和数据块的2维元组序列
for name,group in df.groupby('key1'):
    print(name),print(group)

for (k1,k2),group in df.groupby(['key1','key2']):
    print((k1,k2)),print(group)

a
  key1 key2     data1     data2
0    a  one -0.471537  0.253572
1    a  two  0.894465 -0.633430
4    a  one  0.712625 -1.377129
b
  key1 key2     data1     data2
2    b  one  0.952106 -0.386349
3    b  two  0.925075 -1.386301
('a', 'one')
  key1 key2     data1     data2
0    a  one -0.471537  0.253572
4    a  one  0.712625 -1.377129
('a', 'two')
  key1 key2     data1    data2
1    a  two  0.894465 -0.63343
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.952106 -0.386349
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.925075 -1.386301


In [21]:
#当然你也可以用list,dict的形式把groupby分组后的数据表示出来,选择你想要操作的分组块
dict(list(df.groupby('key1')))

{'a':   key1 key2     data1     data2
 0    a  one -0.471537  0.253572
 1    a  two  0.894465 -0.633430
 4    a  one  0.712625 -1.377129,
 'b':   key1 key2     data1     data2
 2    b  one  0.952106 -0.386349
 3    b  two  0.925075 -1.386301}

In [28]:
#也可以对数据dtype进行分组
for dtype,group in df.groupby(df.dtypes,axis=1):
    print(dtype),print(group)

float64
      data1     data2
0 -0.471537  0.253572
1  0.894465 -0.633430
2  0.952106 -0.386349
3  0.925075 -1.386301
4  0.712625 -1.377129
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


#### 使用字典和series分组

In [30]:
people=pd.DataFrame(np.random.randn(5,5),
                   columns=['a','b','c','d','e'],
                   index=['Joe','Steve','Wes','Jim','Travis'])
people

,a,b,c,d,e
Joe,0.386142,-1.635865,0.600986,0.619721,-0.370000
Steve,0.032962,0.066834,-0.788720,1.770921,0.977503
Wes,1.668766,-1.572035,-0.196553,1.202568,0.731878
Jim,0.671634,-1.242976,-0.285536,-0.011319,-0.312028
Travis,-0.584245,-0.878571,-0.626811,1.860948,-0.285648


In [35]:
#外部自定义一个dict,用字典的值进行分组
mapping={'a':'red','b':'red','c':'blue','d':'blue','e':'red','f':'orange'}
by_column=people.groupby(mapping,axis=1)#通过列索引划分
for i,j in by_column:
    print(i),print(j)
by_column.sum()#对不同分组下进行聚合

blue
               c         d
Joe     0.600986  0.619721
Steve  -0.788720  1.770921
Wes    -0.196553  1.202568
Jim    -0.285536 -0.011319
Travis -0.626811  1.860948
red
               a         b         e
Joe     0.386142 -1.635865 -0.370000
Steve   0.032962  0.066834  0.977503
Wes     1.668766 -1.572035  0.731878
Jim     0.671634 -1.242976 -0.312028
Travis -0.584245 -0.878571 -0.285648


,blue,red
Joe,1.220707,-1.619723
Steve,0.982201,1.077299
Wes,1.006016,0.828609
Jim,-0.296856,-0.883370
Travis,1.234137,-1.748464


In [38]:
#外部自定义series进行分组
map_series=pd.Series(mapping)
people.groupby(map_series,axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,2,3
Jim,2,3
Travis,2,3


#### 使用函数分组

In [44]:
#对行索引的字符串个数进行分组
people.groupby(len).sum()#更加简单
people.groupby([len(i) for i in people.index]).sum()

#将函数与list混合分组
key_list=['one','one','one','two','two']
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one  0.386142 -1.635865 -0.196553  0.619721 -0.370000
  two  0.671634 -1.242976 -0.285536 -0.011319 -0.312028
5 one  0.032962  0.066834 -0.788720  1.770921  0.977503
6 two -0.584245 -0.878571 -0.626811  1.860948 -0.285648

#### 根据索引层级分组

In [49]:
#将MultiIndex.from_array将array类型转化为MultiIndex类型
columns=pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                                  [1,3,5,1,3]],
                                  names=['cty','tenor'])
hier_df=pd.DataFrame(np.random.randn(4,5),columns=columns)
hier_df.groupby(level='cty',axis=1).count()#对多少列计数

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


### 数据聚合（agg和apply映射函数都可)

In [56]:
def peak_to_peak(arr):
    return arr.max()-arr.min()
df.groupby('key1').agg(peak_to_peak)
df.groupby('key1')[['data1','data2']].apply(peak_to_peak)#apply函数无法自动去除object等类型字段

,data1,data2
key1,,
a,1.366002,1.630701
b,0.027031,0.999952


#### 逐列及多函数应用

In [59]:
import seaborn
tips=seaborn.load_dataset('tips')
tips['tip_pct']=tips['tip']/tips['total_bill']
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [69]:
#调用多个函数
tips.groupby(['day','smoker'])[['tip_pct','total_bill']].agg(['mean','std',peak_to_peak])#传入一个List，进行多个聚合

tip_pct                        total_bill             \
                 mean       std peak_to_peak       mean        std   
day  smoker                                                          
Thur Yes     0.163863  0.039389     0.151240  19.190588   8.355149   
     No      0.160298  0.038774     0.193350  17.113111   7.721728   
Fri  Yes     0.174783  0.051293     0.159925  16.813333   9.086388   
     No      0.151650  0.028123     0.067349  18.420000   5.059282   
Sat  Yes     0.147906  0.061375     0.290095  21.276667  10.069138   
     No      0.158048  0.039767     0.235193  19.661778   8.939181   
Sun  Yes     0.187250  0.154134     0.644685  24.120000  10.442511   
     No      0.160113  0.042347     0.193226  20.506667   8.130189   

                          
            peak_to_peak  
day  smoker               
Thur Yes           32.77  
     No            33.68  
Fri  Yes           34.42  
     No            10.29  
Sat  Yes           47.74  
     No            41.08  
Sun  Yes           38.10  
     No            39.40

In [100]:
#可以传入自定义函数名称的元组列表
ftuples=[('Durchschnitt','mean'),('Abweichung',np.var)]#元组中，前者是name，后者是function
tips.groupby(['day','smoker']).agg(ftuples)
#如果是dict来传递修改函数的name，不得行

total_bill                      tip                    size  \
            Durchschnitt  Abweichung Durchschnitt Abweichung Durchschnitt   
day  smoker                                                                 
Thur Yes       19.190588   69.808518     3.030000   1.239863     2.352941   
     No        17.113111   59.625081     2.673778   1.645997     2.488889   
Fri  Yes       16.813333   82.562438     2.714000   1.161369     2.066667   
     No        18.420000   25.596333     2.812500   0.807292     2.250000   
Sat  Yes       21.276667  101.387535     2.875476   2.658791     2.476190   
     No        19.661778   79.908965     3.102889   2.696453     2.555556   
Sun  Yes       24.120000  109.046044     3.516842   1.590501     2.578947   
     No        20.506667   66.099980     3.167895   1.500099     2.929825   

                            tip_pct             
            Abweichung Durchschnitt Abweichung  
day  smoker                                     
Thur Yes      0.492647     0.163863   0.001551  
     No       1.391919     0.160298   0.001503  
Fri  Yes      0.352381     0.174783   0.002631  
     No       0.250000     0.151650   0.000791  
Sat  Yes      0.743322     0.147906   0.003767  
     No       0.616162     0.158048   0.001581  
Sun  Yes      0.812865     0.187250   0.023757  
     No       1.066416     0.160113   0.001793

In [77]:
#将不同函数应用到不同列上,可以传入一个dict
tips.groupby(['day','smoker']).agg({'tip':np.max,'size':'sum'})#tip这一列找最大值，size这一列加总

#一个列应用多个函数
tips.groupby(['day','smoker']).agg({'tip_pct':['min','max','mean','std'],
                                  'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Thur Yes     0.090014  0.241255  0.163863  0.039389   40
     No      0.072961  0.266312  0.160298  0.038774  112
Fri  Yes     0.103555  0.263480  0.174783  0.051293   31
     No      0.120385  0.187735  0.151650  0.028123    9
Sat  Yes     0.035638  0.325733  0.147906  0.061375  104
     No      0.056797  0.291990  0.158048  0.039767  115
Sun  Yes     0.065660  0.710345  0.187250  0.154134   49
     No      0.059447  0.252672  0.160113  0.042347  167

#### 返回不含行索引的聚合数据

In [80]:
tips.groupby(['day','smoker'],as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Thur,Yes,19.190588,3.030000,2.352941,0.163863
1,Thur,No,17.113111,2.673778,2.488889,0.160298
2,Fri,Yes,16.813333,2.714000,2.066667,0.174783
3,Fri,No,18.420000,2.812500,2.250000,0.151650
4,Sat,Yes,21.276667,2.875476,2.476190,0.147906
5,Sat,No,19.661778,3.102889,2.555556,0.158048
6,Sun,Yes,24.120000,3.516842,2.578947,0.187250
7,Sun,No,20.506667,3.167895,2.929825,0.160113


### 拆分-应用-联合

In [90]:
#传入一个函数
def top(df,n=5,column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips,n=6)#调用

tips.groupby('smoker').apply(top)#分组后，用apply调用
tips.groupby(['smoker','day']).apply(top,n=1,column='total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
Yes    Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   
       Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
No     Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
       Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   

                  tip_pct  
smoker day                 
Yes    Thur 197  0.115982  
       Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
No     Thur 142  0.121389  
       Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799

#### 压缩分组键

In [92]:
#可以通过group_keys来拒绝分组键的出现
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990


#### 分位数与桶分析
cut与qcut

In [95]:
frame=pd.DataFrame({'data1':np.random.randn(1000),
                   'data2':np.random.randn(1000)})
quartiles=pd.cut(frame.data1,4)#4个距离间距类型
quartiles

0       (-0.561, 1.197]
1       (-0.561, 1.197]
2       (-0.561, 1.197]
3       (-0.561, 1.197]
4        (1.197, 2.954]
             ...       
995    (-2.318, -0.561]
996     (-0.561, 1.197]
997     (-0.561, 1.197]
998    (-2.318, -0.561]
999     (-0.561, 1.197]
Name: data1, Length: 1000, dtype: category
Categories (4, interval[float64]): [(-4.083, -2.318] < (-2.318, -0.561] < (-0.561, 1.197] < (1.197, 2.954]]

In [108]:
#用字典来修改函数名称
def get_stats(group):
    return {'min':group.min(),'max':group.max(),'count':group.count(),'mean':group.mean()}
frame.data2.groupby(quartiles).apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-4.083, -2.318]",-2.081819,1.600475,10.0,0.099954
"(-2.318, -0.561]",-2.893872,2.523995,310.0,-0.011647
"(-0.561, 1.197]",-3.176622,3.175372,574.0,0.030611
"(1.197, 2.954]",-2.666278,2.643901,106.0,0.070673


In [105]:
frame.data2.groupby(pd.cut(frame.data1,4)).agg(['min','max','count','mean'])#一样的结果

,min,max,count,mean
data1,,,,
"(-4.083, -2.318]",-2.081819,1.600475,10,0.099954
"(-2.318, -0.561]",-2.893872,2.523995,310,-0.011647
"(-0.561, 1.197]",-3.176622,3.175372,574,0.030611
"(1.197, 2.954]",-2.666278,2.643901,106,0.070673


In [111]:
#qcut分位数来分
frame.data2.groupby(pd.qcut(frame.data1,10)).apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-4.077, -1.318]",-2.432259,2.147230,100.0,-0.040609
"(-1.318, -0.913]",-1.987030,2.523995,100.0,-0.001245
"(-0.913, -0.614]",-2.893872,2.177258,100.0,0.018426
"(-0.614, -0.339]",-2.470976,2.014057,100.0,0.070400
"(-0.339, -0.0745]",-2.438390,3.175372,100.0,0.120408
"(-0.0745, 0.164]",-3.176622,2.287979,100.0,-0.155939
"(0.164, 0.437]",-2.215279,2.346012,100.0,-0.026092
"(0.437, 0.741]",-2.247589,2.819166,100.0,0.180756
"(0.741, 1.224]",-2.295219,2.361917,100.0,-0.022739


#### 指定分组值填充缺失值

In [112]:
#fillna即可填充
#groupby前后填充的方式

#### 随机采样与排列(扑克牌分发问题)
<span class="mark">groupby下面是没有sort_values,所以用自定义的apply(lambda x:x.sort_values(by=''))方法实现</span>

In [337]:
#1.创造牌
types=['黑桃','红桃','梅花','方片']
num=['A']+[str(i) for i in range(2,11)]+['J','Q','K']
standard=[]
lists=[]#用来整理牌的号
for color in types:
    for i,nums in enumerate(num,start=1):
        standard.append(color+nums)
        lists.append(i)
df=pd.DataFrame({'每个花色下牌顺序':lists,'扑克牌':standard})
df1=df.copy()

#2.洗牌分发
df1['洗牌序号']=np.random.permutation(52)#洗牌
df1.sort_values(by='洗牌序号',inplace=True)
df1.drop('洗牌序号',axis=1,inplace=True)
people=[]
for i in range(4):
    people.append(df1[i:52:4])#抽牌
    
#3.每个人将自己手中的牌进行整理
player=[]
for i in range(len(people)):
    player.append(people[i].groupby(people[i].loc[:,'扑克牌'].str[:2],as_index=False).apply(lambda x : x.sort_values('每个花色下牌顺序')).reset_index(drop=True))
df2=pd.DataFrame()
name=['小明','小红','小谢','小白']
for i,j in enumerate(name):
    df2[j]=player[i]['扑克牌']
df2

,小明,小红,小谢,小白
0,方片A,方片2,方片5,方片8
1,方片3,方片Q,方片6,方片J
2,方片4,梅花A,方片7,梅花J
3,方片10,梅花2,方片9,红桃3
4,方片K,梅花7,梅花3,红桃5
5,梅花4,梅花9,梅花8,红桃6
6,梅花5,梅花10,红桃7,红桃9
7,梅花6,梅花Q,黑桃2,红桃Q
8,梅花K,红桃A,黑桃3,红桃K
9,红桃4,红桃2,黑桃8,黑桃6


#### 分组加权平均和相关性

In [173]:
#针对average这个函数，赋权重
df=pd.DataFrame({'category':['a','a','a','a','b','b','b','b'],
                'data':np.random.randn(8),
                'weights':np.random.rand(8)})
df

,category,data,weights
0,a,-0.240856,0.551262
1,a,1.182514,0.768242
2,a,1.725465,0.764104
3,a,1.905499,0.303550
4,b,-1.216824,0.467863
5,b,0.233922,0.161927
6,b,0.475401,0.746677
7,b,1.279426,0.500623


In [174]:
df.groupby('category').apply(lambda x : np.average(x['data'],weights=x['weights']))

category
a    1.119545
b    0.247219
dtype: float64

In [181]:
from sklearn.datasets import load_boston
model=load_boston()
data=pd.DataFrame(model.data,columns=model.feature_names)
data.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [198]:
#corrwith()函数：corrwith用于计算DataFrame中行与行或者列与列之间的相关性
data.corrwith(data['B'])
data.LSTAT.corr(data['B'])#corr用于计算Series之间的相关性，是corrwith的最小单元。
data.corr()#整体的相关性

#data.groupby('RAD').apply(lambda x:x.corrwith(data['B']))#还可以分组在进行聚合求相关性

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
CRIM,1.000000,-0.200469,0.406583,-0.055892,0.420972,-0.219247,0.352734,-0.379670,0.625505,0.582764,0.289946,-0.385064,0.455621
ZN,-0.200469,1.000000,-0.533828,-0.042697,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.175520,-0.412995
INDUS,0.406583,-0.533828,1.000000,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.720760,0.383248,-0.356977,0.603800
CHAS,-0.055892,-0.042697,0.062938,1.000000,0.091203,0.091251,0.086518,-0.099176,-0.007368,-0.035587,-0.121515,0.048788,-0.053929
NOX,0.420972,-0.516604,0.763651,0.091203,1.000000,-0.302188,0.731470,-0.769230,0.611441,0.668023,0.188933,-0.380051,0.590879
RM,-0.219247,0.311991,-0.391676,0.091251,-0.302188,1.000000,-0.240265,0.205246,-0.209847,-0.292048,-0.355501,0.128069,-0.613808
AGE,0.352734,-0.569537,0.644779,0.086518,0.731470,-0.240265,1.000000,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339
DIS,-0.379670,0.664408,-0.708027,-0.099176,-0.769230,0.205246,-0.747881,1.000000,-0.494588,-0.534432,-0.232471,0.291512,-0.496996
RAD,0.625505,-0.311948,0.595129,-0.007368,0.611441,-0.209847,0.456022,-0.494588,1.000000,0.910228,0.464741,-0.444413,0.488676
TAX,0.582764,-0.314563,0.720760,-0.035587,0.668023,-0.292048,0.506456,-0.534432,0.910228,1.000000,0.460853,-0.441808,0.543993


In [192]:
data.pct_change()#求一个百分比变动，第一行会空缺
data.pct_change().dropna()#删除第一行

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
269,-0.832285,0.0,0.753149,inf,-0.193043,-0.207497,0.169202,0.364032,-0.400000,-0.155303,0.430769,0.002665,3.319620
270,2.300165,0.0,0.000000,-1.0,0.000000,-0.010811,-0.315447,0.130568,0.000000,0.000000,0.000000,-0.006874,-0.047619
273,0.936126,0.0,0.000000,inf,0.000000,0.176354,-0.117547,0.114614,0.000000,0.000000,0.000000,-0.010609,-0.148771
274,-0.745628,1.0,-0.079023,0.0,-0.036638,-0.121311,-0.364865,-0.066163,0.333333,0.139013,-0.053763,0.015687,-0.463526
275,0.701630,0.0,0.000000,-1.0,0.000000,0.014205,0.300912,0.046522,0.000000,0.000000,0.000000,0.000000,-0.155807
276,0.090067,0.0,0.000000,inf,0.000000,0.060257,0.144860,0.121833,0.000000,0.000000,0.000000,-0.019274,1.030201
277,-0.414748,0.0,0.000000,0.0,0.000000,-0.060685,-0.436735,0.015792,0.000000,0.000000,0.000000,0.010790,-0.312397
278,0.302105,0.0,0.000000,-1.0,0.000000,-0.050396,0.163043,-0.148577,0.000000,0.000000,0.000000,0.008769,0.728365
282,0.654251,0.0,0.000000,inf,0.000000,0.097158,0.336022,-0.006254,0.000000,0.000000,0.000000,-0.038651,-0.344227
283,-0.755099,3.5,-0.636637,0.0,-0.094604,0.036364,-0.501006,0.129147,-0.800000,-0.083333,-0.087248,0.048930,0.049834


### 数据透视表与交叉表

1. <span class="mark">pivot：无法聚合，只能简单重塑</span>，如果存在<span class="burk">重复数据将会报错</span>；常用于处理非数字数据。
2. <span class="burk">pivot_table：可以聚合</span>，正好弥补 pivot 的缺陷。默认：<span class="burk">mean</span>

<span class="mark">平时正常用一般是pivot ，只有在类似groupby分组下，才会用pivot_table</span>

<span class="mark">如果分组中数据有重复，就无法使用pivot了，比如time作为index，day作为columns，当在数据中有重复的time和day组合，就会报错</span>

In [199]:
tips.pivot_table('tip_pct',index=['time','day'],columns='smoker')#默认mean聚合

smoker            Yes        No
time   day                     
Lunch  Thur  0.163863  0.160311
       Fri   0.188937  0.187735
Dinner Thur       NaN  0.159744
       Fri   0.165347  0.139622
       Sat   0.147906  0.158048
       Sun   0.187250  0.160113

In [200]:
#增加margins=True这个代码，扩充这个表，增加All行和列的标签
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker',margins=True)

size                       tip_pct                    
smoker            Yes        No       All       Yes        No       All
time   day                                                             
Lunch  Thur  2.352941  2.500000  2.459016  0.163863  0.160311  0.161301
       Fri   1.833333  3.000000  2.000000  0.188937  0.187735  0.188765
Dinner Thur       NaN  2.000000  2.000000       NaN  0.159744  0.159744
       Fri   2.222222  2.000000  2.166667  0.165347  0.139622  0.158916
       Sat   2.476190  2.555556  2.517241  0.147906  0.158048  0.153152
       Sun   2.578947  2.929825  2.842105  0.187250  0.160113  0.166897
All          2.408602  2.668874  2.569672  0.163196  0.159328  0.160803

In [228]:
#还可以更改aggfunc
tips.pivot_table('tip_pct',index=['time','smoker'],columns='day',aggfunc=len,margins=True)#频率计数
#如果出现空缺值，则增加fill_value=0即可

day            Thur   Fri   Sat   Sun    All
time   smoker                               
Lunch  Yes     17.0   6.0   NaN   NaN   23.0
       No      44.0   1.0   NaN   NaN   45.0
Dinner Yes      NaN   9.0  42.0  19.0   70.0
       No       1.0   3.0  45.0  57.0  106.0
All            62.0  19.0  87.0  76.0  244.0

#### 交叉表crosstab
计算分组后的频率

In [230]:
pd.crosstab(tips.smoker,tips.day,margins=True)#单个
pd.crosstab([tips.smoker,tips.time],tips.day,margins=True)

day            Thur  Fri  Sat  Sun  All
smoker time                            
Yes    Lunch     17    6    0    0   23
       Dinner     0    9   42   19   70
No     Lunch     44    1    0    0   45
       Dinner     1    3   45   57  106
All              62   19   87   76  244

## 高阶pandas

In [5]:
#用take方法可以恢复重复列的值
values=pd.Series([0,1,0,0]*2)
dim=pd.Series(['apple','orange'])
dim.take(values)#dim作为类似字典的值，values作为编码code，即key 

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### categorical类型

In [8]:
import numpy as np
df=pd.DataFrame({'fruit':['apple','orange','apple','apple']*2,
                'basket_id':np.arange(8),
                'count':np.random.randint(3,15,size=8),
                'weight':np.random.uniform(0,4,8)},
               columns=['basket_id','fruit','count','weight'])
fruit_cat=df['fruit'].astype('category')

In [15]:
c=fruit_cat.values
c.categories#查询unique后的值
c.codes#查询每个值的编码
df['fruit']=df['fruit'].astype('category')#转为category
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

#### 直接建立category

In [16]:
pd.Categorical(['foo','barz','barz','foo','bar'])

['foo', 'barz', 'barz', 'foo', 'bar']
Categories (3, object): ['bar', 'barz', 'foo']

In [22]:
#用code创建categorical类型
my_cats_2=pd.Categorical.from_codes([0,1,2,0,0,1],['foo','bar','baz'])
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [20]:
#加上内部排序条件
pd.Categorical.from_codes([0,1,2,0,0,1],['foo','bar','baz'],ordered=True)

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [25]:
#用as_ordered排序
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

#### 使用categorical对象进行计算

In [31]:
draws=np.random.randn(1000)
bins=pd.qcut(draws,4,labels=['Q1','Q2','Q3','Q4'])
pd.Series(draws).groupby(bins).agg(['count','min','max'])#通过category类型来进行分组

,count,min,max
Q1,250,-3.389250,-0.634565
Q2,250,-0.629451,0.045374
Q3,250,0.046756,0.676663
Q4,250,0.681453,2.860714


### 针对groupby后的理解（transform）

In [34]:
df=pd.DataFrame({'key':['a','b','c']*4,
                'value':np.arange(12.)})
df.head()

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0


In [37]:
g=df.groupby('key').value
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [40]:
#transform作用于所有元素,两个是一样的
g.transform(lambda x:x.mean())
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [43]:
#transformz作用于函数，进行标准化
def normalize(x):
    return(x-x.mean())/x.std()
g.transform(normalize)
g.apply(normalize)#也是一样的结果

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

### <span class="mark">resample来时间分组</span>

In [47]:
N=15
times=pd.date_range('2007-05-20 00:00',freq='1min',periods=N)
df=pd.DataFrame({'time':times,'value':np.arange(N)})
df.head()

,time,value
0,2007-05-20 00:00:00,0
1,2007-05-20 00:01:00,1
2,2007-05-20 00:02:00,2
3,2007-05-20 00:03:00,3
4,2007-05-20 00:04:00,4


In [52]:
df.set_index('time').resample('5min').count()#以5分钟为一组

,value
time,
2007-05-20 00:00:00,5
2007-05-20 00:05:00,5
2007-05-20 00:10:00,5
